In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autism_spectrum_disorder_(ASD)"
cohort = "GSE57802"

# Input paths
in_trait_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)"
in_cohort_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE57802"

# Output paths
out_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE57802.csv"
out_gene_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE57802.csv"
out_clinical_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE57802.csv"
json_path = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome Profiling of patients with 16p11.2 rearrangements"
!Series_summary	"The 600kb BP4-BP5 16p11.2 CNV (copy number variant) is associated with neuroanatomical, neurocognitive and metabolic disorders.  These recurrent rearrangements are associated with reciprocal phenotypes such as obesity and underweight, macro- and microcephaly, as well as autism spectrum disorder (ASD) and schizophrenia. Here we interrogated the transcriptome of individuals carrying reciprocal CNVs in 16p11.2."
!Series_summary	"The genome-wide transcript perturbations correlated with clinical endophenotypes of the CNV and were enriched for genes associated with ASD. We uncovered a significant correlation between copy number changes and expression levels of genes mutated in ciliopathies."
!Series_overall_design	"Transcriptome profiles of lymphoblastoid cell lines of 50 16p11.2 deletion carriers, 31 16p11.2 duplication carriers and 17 controls."
Sample Characteristics Di

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Transcriptome profiling implies gene expression data is available

# 2. Variable Availability and Data Type Conversion
# Note from background info that the dataset is about 16p11.2 CNV carriers with some having ASD

# 2.1 Data Availability
# For trait (ASD), we don't have a direct row, but we can use the genotype information
trait_row = 4  # 'genotype: Control', 'genotype: 600kbdel', 'genotype: 600kbdup'
age_row = 2  # 'age: X' values
gender_row = 1  # 'gender: M', 'gender: F'

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert genotype information to binary trait values.
    Based on the background information, 16p11.2 deletions and duplications are associated with ASD.
    - genotype: Control (0) - control subjects
    - genotype: 600kbdel (1) - deletion carriers, associated with ASD
    - genotype: 600kbdup (1) - duplication carriers, associated with ASD
    """
    if not value or ':' not in value:
        return None
    genotype = value.split(':', 1)[1].strip().lower()
    if 'control' in genotype:
        return 0  # Control
    elif '600kbdel' in genotype or '600kbdup' in genotype:
        return 1  # CNV carriers (associated with ASD)
    return None

def convert_age(value):
    """Convert age values to continuous numeric values."""
    if not value or ':' not in value:
        return None
    age_str = value.split(':', 1)[1].strip()
    if age_str.lower() == 'na':
        return None
    try:
        return float(age_str)
    except:
        return None

def convert_gender(value):
    """
    Convert gender values to binary:
    - F (female) = 0
    - M (male) = 1
    """
    if not value or ':' not in value:
        return None
    gender = value.split(':', 1)[1].strip().upper()
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save clinical data to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM1389621': [0.0, 46.0, 1.0], 'GSM1389622': [0.0, 33.0, 0.0], 'GSM1389623': [0.0, nan, 1.0], 'GSM1389624': [0.0, nan, 0.0], 'GSM1389625': [0.0, 22.0, 1.0], 'GSM1389626': [0.0, 52.0, 1.0], 'GSM1389627': [0.0, 25.0, 1.0], 'GSM1389628': [0.0, 31.0, 0.0], 'GSM1389629': [0.0, 60.0, 1.0], 'GSM1389630': [0.0, nan, 1.0], 'GSM1389631': [0.0, 40.0, 1.0], 'GSM1389632': [0.0, 50.0, 1.0], 'GSM1389633': [0.0, 51.0, 1.0], 'GSM1389634': [0.0, 39.0, 1.0], 'GSM1389635': [0.0, 6.0, 1.0], 'GSM1389636': [0.0, 51.0, 1.0], 'GSM1389637': [0.0, 56.0, 0.0], 'GSM1389638': [1.0, 16.0, 1.0], 'GSM1389639': [1.0, 41.0, 1.0], 'GSM1389640': [1.0, 31.0, 0.0], 'GSM1389641': [1.0, 35.0, 1.0], 'GSM1389642': [1.0, 4.0, 1.0], 'GSM1389643': [1.0, 10.0, 0.0], 'GSM1389644': [1.0, 12.0, 0.0], 'GSM1389645': [1.0, 7.0, 1.0], 'GSM1389646': [1.0, 6.0, 1.0], 'GSM1389647': [1.0, 1.4, 1.0], 'GSM1389648': [1.0, 10.0, 0.0], 'GSM1389649': [1.0, 6.0, 1.0], 'GSM1389650': [1.0, 38.0, 1.0], 'GSM1389651': [1.0, 14.7,

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the expression data
# These appear to be Affymetrix probe IDs (with the "PM" format and "_at" suffixes)
# rather than standard human gene symbols like BRCA1, TP53, etc.
# Affymetrix probe IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the annotation data contain probe IDs and gene symbols
# From the preview, 'ID' contains probe IDs like '1007_PM_s_at' matching the expression data IDs
# 'Gene Symbol' contains standard gene symbols like 'DDR1'

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)

# Print the first few gene symbols after mapping to verify
print("First 20 gene symbols after mapping:")
print(gene_data.index[:20])


First 20 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC', 'AADACL2', 'AADAT', 'AAGAB',
       'AAK1', 'AAMP', 'AANAT', 'AARS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. We need to first create the selected_clinical_df using clinical_data from Step 1
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the extracted clinical features
preview = preview_df(selected_clinical_df)
print("Preview of selected clinical features:")
print(preview)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and demographic features are severely biased
trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
if trait_type == 'binary':
    is_trait_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait)

# Remove biased demographic features
unbiased_linked_data = linked_data.copy()
if 'Age' in unbiased_linked_data.columns:
    age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
    if age_biased:
        unbiased_linked_data = unbiased_linked_data.drop(columns=['Age'])
        
if 'Gender' in unbiased_linked_data.columns:
    gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
    if gender_biased:
        unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data, 
    note="Dataset contains gene expression data from iPSC-derived neurons of ASD patients and unaffected siblings."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("The dataset was determined to be not usable for analysis.")

Preview of selected clinical features:
{'GSM1389621': [0.0, 46.0, 1.0], 'GSM1389622': [0.0, 33.0, 0.0], 'GSM1389623': [0.0, nan, 1.0], 'GSM1389624': [0.0, nan, 0.0], 'GSM1389625': [0.0, 22.0, 1.0], 'GSM1389626': [0.0, 52.0, 1.0], 'GSM1389627': [0.0, 25.0, 1.0], 'GSM1389628': [0.0, 31.0, 0.0], 'GSM1389629': [0.0, 60.0, 1.0], 'GSM1389630': [0.0, nan, 1.0], 'GSM1389631': [0.0, 40.0, 1.0], 'GSM1389632': [0.0, 50.0, 1.0], 'GSM1389633': [0.0, 51.0, 1.0], 'GSM1389634': [0.0, 39.0, 1.0], 'GSM1389635': [0.0, 6.0, 1.0], 'GSM1389636': [0.0, 51.0, 1.0], 'GSM1389637': [0.0, 56.0, 0.0], 'GSM1389638': [1.0, 16.0, 1.0], 'GSM1389639': [1.0, 41.0, 1.0], 'GSM1389640': [1.0, 31.0, 0.0], 'GSM1389641': [1.0, 35.0, 1.0], 'GSM1389642': [1.0, 4.0, 1.0], 'GSM1389643': [1.0, 10.0, 0.0], 'GSM1389644': [1.0, 12.0, 0.0], 'GSM1389645': [1.0, 7.0, 1.0], 'GSM1389646': [1.0, 6.0, 1.0], 'GSM1389647': [1.0, 1.4, 1.0], 'GSM1389648': [1.0, 10.0, 0.0], 'GSM1389649': [1.0, 6.0, 1.0], 'GSM1389650': [1.0, 38.0, 1.0], 'GSM13896

Normalized gene data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE57802.csv


For the feature 'Autism_spectrum_disorder_(ASD)', the least common label is '0.0' with 17 occurrences. This represents 17.17% of the dataset.
Quartiles for 'Age':
  25%: 6.4
  50% (Median): 14.7
  75%: 36.5
Min: 0.6
Max: 76.0
For the feature 'Gender', the least common label is '0.0' with 41 occurrences. This represents 41.41% of the dataset.


Linked data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE57802.csv
